In [20]:
%load_ext rpy2.ipython

In [21]:
%%R -o gene2name
suppressMessages(suppressWarnings(library(tidyverse)))

GTF = '~/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf'

gtf <- rtracklayer::import(GTF)
gene2name <- gtf[gtf$type == "gene"] %>% data.frame %>% column_to_rownames('gene_id') %>% dplyr::select('gene_name')

In [151]:
import pandas as pd

import numpy as np

> ### event-level analysis
you can also do an event-level analysis
where you use the exon-level results
but as genefile use ChIP-seq data intersected with ex1-ex2-ex3 event (basically upstrem and downstream exon included with the intron in the middle)

> for exon-level analysus
`JunctionSeqallGenes.results.txt.gz`
has the exon coordinates `chr	start	end	strand`
also has the transcripts that exon junction is in transcripts
you can go to gff flatfile and find exons upstream and downstream of the featyre at hand

In [119]:
def read_results_table(PATH):
    '''event-level analysis
    '''
    table = pd.read_csv(PATH,sep='\t').drop("featureID",axis=1).dropna(subset=['expr_dox']).dropna(axis=1)
    # .drop("featureID",axis=1)
    table = pd.concat([
        gene2name.loc[table.geneID,:].reset_index(drop=True),
        table.reset_index(drop=True)
    ],axis=1).set_index(["geneID","gene_name","countbinID"])
    
    table=table[[i for i in table if len(set(table[i]))>1]]

    return table

In [120]:
sigtable = read_results_table('jscs/sigGenes.results.txt.gz')

In [269]:
sigtable

baseMean      baseVar  \
geneID             gene_name countbinID                            
ENSG00000116288.13 PARK7     E003         89.500235   403.511739   
                             E004        107.180418   981.856850   
                             E005        195.340298   925.603565   
                             E006         95.757555   353.996437   
                             E007        227.881779  1930.523043   
...                                             ...          ...   
ENSG00000147140.16 NONO      J066        652.632118  4788.507214   
                             J067        609.522401  2370.891868   
                             J068        683.650367  1177.148546   
                             J069        727.299294    52.267422   
                             J070        682.227425   662.603772   

                                         dispBeforeSharing  dispFitted  \
geneID             gene_name countbinID                                  
ENSG00000116288.13 PARK7     E003                 0.042550    0.012923   
                             E004                 0.056942    0.011358   
                             E005                 0.016472    0.007782   
                             E006                 0.013577    0.012303   
                             E007                 0.000395    0.007162   
...                                                    ...         ...   
ENSG00000147140.16 NONO      J066                 0.000830    0.003741   
                             J067                 0.001440    0.003823   
                             J068                 0.000024    0.003688   
                             J069                 0.000052    0.003622   
                             J070                 0.000014    0.003690   

                                         dispersion    pvalue   padjust   chr  \
geneID             gene_name countbinID                                         
ENSG00000116288.13 PARK7     E003          0.033333  0.788027  0.988488  chr1   
                             E004          0.042257  0.554762  0.961012  chr1   
                             E005          0.013927  0.285620  0.878576  chr1   
                             E006          0.013202  0.139371  0.762814  chr1   
                             E007          0.002474  0.005040  0.245486  chr1   
...                                             ...       ...       ...   ...   
ENSG00000147140.16 NONO      J066          0.001521  0.071776  0.642267  chrX   
                             J067          0.001992  0.828729  0.991763  chrX   
                             J068          0.000873  0.300250  0.887474  chrX   
                             J069          0.000920  0.339330  0.904853  chrX   
                             J070          0.000815  0.948915  0.998149  chrX   

                                            start       end strand  \
geneID             gene_name countbinID                              
ENSG00000116288.13 PARK7     E003         7961691   7961700      +   
                             E004         7961700   7961710      +   
                             E005         7961710   7961735      +   
                             E006         7961735   7961747      +   
                             E007         7961747   7961793      +   
...                                           ...       ...    ...   
ENSG00000147140.16 NONO      J066        71297047  71297376      +   
                             J067        71297461  71297835      +   
                             J068        71297938  71298468      +   
                             J069        71298508  71298706      +   
                             J070        71298816  71299941      +   

                                                                               transcripts  \
geneID             gene_name countbinID                                                      
ENSG00000116288.13 PARK7     E003         

In [122]:
len(set(sigtable.index.to_frame().geneID))

71

In [270]:
sigtable.head()

baseMean      baseVar  \
geneID             gene_name countbinID                            
ENSG00000116288.13 PARK7     E003         89.500235   403.511739   
                             E004        107.180418   981.856850   
                             E005        195.340298   925.603565   
                             E006         95.757555   353.996437   
                             E007        227.881779  1930.523043   

                                         dispBeforeSharing  dispFitted  \
geneID             gene_name countbinID                                  
ENSG00000116288.13 PARK7     E003                 0.042550    0.012923   
                             E004                 0.056942    0.011358   
                             E005                 0.016472    0.007782   
                             E006                 0.013577    0.012303   
                             E007                 0.000395    0.007162   

                                         dispersion    pvalue   padjust   chr  \
geneID             gene_name countbinID                                         
ENSG00000116288.13 PARK7     E003          0.033333  0.788027  0.988488  chr1   
                             E004          0.042257  0.554762  0.961012  chr1   
                             E005          0.013927  0.285620  0.878576  chr1   
                             E006          0.013202  0.139371  0.762814  chr1   
                             E007          0.002474  0.005040  0.245486  chr1   

                                           start      end strand  \
geneID             gene_name countbinID                            
ENSG00000116288.13 PARK7     E003        7961691  7961700      +   
                             E004        7961700  7961710      +   
                             E005        7961710  7961735      +   
                             E006        7961735  7961747      +   
                             E007        7961747  7961793      +   

                                                                               transcripts  \
geneID             gene_name countbinID                                                      
ENSG00000116288.13 PARK7     E003                      ENST00000460192.5;ENST00000493678.5   
                             E004        ENST00000460192.5;ENST00000493678.5;ENST000003...   
                             E005        ENST00000460192.5;ENST00000493678.5;ENST000003...   
                             E006                                       ENST00000338639.10   
                             E007                     ENST00000338639.10;ENST00000465354.5   

                                         featureType  padjust_noFilter  \
geneID             gene_name countbinID                                  
ENSG00000116288.13 PARK7     E003        exonic_part          0.992016   
                             E004        exonic_part          0.980430   
                             E005        exonic_part          0.980430   
                             E006        exonic_part          0.908534   
                             E007        exonic_part          0.387210   

                                         log2FC(dox/no-dox)  \
geneID             gene_name countbinID                       
ENSG00000116288.13 PARK7     E003                 -0.137950   
                             E004                 -0.303329   
                             E005                 -0.282505   
                             E006                 -0.411065   
                             E007                 -0.394167   

                                         log2FCvst(dox/no-dox)  expr_no-dox  \
geneID             gene_name countbinID                                       
ENSG00000116288.13 PARK7     E003                    -0.068974    95.017271   
                             E004                    -0.151660   119.706081   
                             E005                    -0.141245   2

In [124]:
alltable = read_results_table('jscs/allGenes.results.txt.gz')

In [271]:
len(set(alltable.index.to_frame().geneID))

8093

In [272]:
alltable.head()

baseMean      baseVar  \
geneID             gene_name countbinID                             
ENSG00000187634.12 SAMD11    E036          93.110387   293.519556   
                             E037         539.210683  5896.530559   
                             E038         351.463314  4337.431386   
                             E039         430.266075  7322.845950   
ENSG00000188976.11 NOC2L     E003        1005.274236  5695.877938   

                                         dispBeforeSharing  dispFitted  \
geneID             gene_name countbinID                                  
ENSG00000187634.12 SAMD11    E036             2.222378e-03    0.012555   
                             E037             8.370035e-03    0.005010   
                             E038             1.000001e-08    0.005852   
                             E039             4.485993e-03    0.005409   
ENSG00000188976.11 NOC2L     E003             8.708665e-04    0.004280   

                                         dispersion    pvalue   padjust   chr  \
geneID             gene_name countbinID                                         
ENSG00000187634.12 SAMD11    E036          0.005819  0.094784  0.692075  chr1   
                             E037          0.006941  0.034466  0.510059  chr1   
                             E038          0.005711  0.525253  0.955419  chr1   
                             E039          0.004843  0.008196  0.295338  chr1   
ENSG00000188976.11 NOC2L     E003          0.001551  0.224609  0.839599  chr1   

                                          start     end strand  \
geneID             gene_name countbinID                          
ENSG00000187634.12 SAMD11    E036        943907  944259      +   
                             E037        944259  944574      +   
                             E038        944574  944575      +   
                             E039        944575  944581      +   
ENSG00000188976.11 NOC2L     E003        944204  944800      -   

                                                                               transcripts  \
geneID             gene_name countbinID                                                      
ENSG00000187634.12 SAMD11    E036        ENST00000618779.4;ENST00000618323.4;ENST000006...   
                             E037        ENST00000618779.4;ENST00000618323.4;ENST000006...   
                             E038        ENST00000618779.4;ENST00000618323.4;ENST000006...   
                             E039        ENST00000618779.4;ENST00000618323.4;ENST000006...   
ENSG00000188976.11 NOC2L     E003        ENST00000327044.7;ENST00000483767.5;ENST000004...   

                                         featureType  padjust_noFilter  \
geneID             gene_name countbinID                                  
ENSG00000187634.12 SAMD11    E036        exonic_part          0.840749   
                             E037        exonic_part          0.670369   
                             E038        exonic_part          0.980430   
                             E039        exonic_part          0.447949   
ENSG00000188976.11 NOC2L     E003        exonic_part          0.980430   

                                         log2FC(dox/no-dox)  \
geneID             gene_name countbinID                       
ENSG00000187634.12 SAMD11    E036                  0.363475   
                             E037                  0.500392   
                             E038                 -0.134919   
                             E039                 -0.504995   
ENSG00000188976.11 NOC2L     E003                  0.117735   

                                         log2FCvst(dox/no-dox)  expr_no-dox  \
geneID             gene_name countbinID                                       
ENSG00000187634.12 SAMD11    E036                     0.181719    90.830636   
                             E037                     0.249254   573.202964   
                             E038                    -0.067408   389.26432

___
> ### gives a gene-level file as well, you can use that. 
to run teiser
with ChIP-seq data intersected with genes as genefile

> this is the gene based file
`JunctionSeqsigGenes.genewiseResults.txt.gz`
genes listed here go to class ‘1’
and all the other expressed genes class ‘0’



In [127]:
def read_genewiseResults_table(PATH):
    '''event-level analysis
    '''
    table = pd.read_csv(PATH,sep='\t')#.drop("featureID",axis=1).dropna(subset=['expr_dox']).dropna(axis=1)
    table = table.set_index(["geneID","geneName"])

    return table

In [135]:
genewise_table = read_genewiseResults_table("jscs/sigGenes.genewiseResults.txt.gz")
genewise_genes = set(genewise_table.index.get_level_values(1))

In [258]:
genewise_table

,,chr,start,end,strand,baseMean,geneWisePadj,mostSigID,mostSigPadjust,numExons,numKnown,numNovel,exonsSig,knownSig,novelSig,numFeatures,numSig
geneID,geneName,,,,,,,,,,,,,,,,
ENSG00000116288.13,PARK7,chr1,7954290,7985505,+,2286.2,0.003250,E011,0.001890,28,15,0,1,0,0,28/15/0,1/0/0
ENSG00000179051.14,RCC2,chr1,17406759,17439677,-,4742.0,0.014966,J032,0.008920,19,13,0,0,1,0,19/13/0,0/1/0
ENSG00000142949.17,PTPRF,chr1,43525186,43623666,+,4391.4,0.003330,E007,0.001930,69,47,0,1,0,0,69/47/0,1/0/0
ENSG00000117450.14,PRDX1,chr1,45511035,45523047,-,8454.0,0.000073,J019,0.000046,15,9,0,0,1,0,15/9/0,0/1/0
ENSG00000135829.17,DHX9,chr1,182839346,182887982,+,4657.1,0.014732,E003,0.008740,44,32,0,1,0,0,44/32/0,1/0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000168453.15,HR,chr8,22114418,22133384,-,1094.6,0.011513,E033,0.006760,35,22,0,1,0,0,35/22/0,1/0/0
ENSG00000147471.12,PLPBP,chr8,37762594,37779768,+,1159.0,0.013891,J027,0.008160,25,10,0,0,1,0,25/10/0,0/1/0
ENSG00000136942.15,RPL35,chr9,124857879,124861981,-,8494.2,0.000084,J021,0.000053,15,6,0,2,1,0,15/6/0,2/1/0


In [129]:
def read_expression_table(PATH):
    '''
    '''
    table = pd.read_csv(PATH,sep='\t').dropna(subset=['expr_dox']).dropna(axis=1)
    table["geneID"] = table.featureID.str.split(':').str[0]
    table["countbinID"] = table.featureID.str.split(':').str[1]
    
    table = pd.concat([
        gene2name.loc[table.geneID,:].reset_index(drop=True),
        table.drop("featureID",axis=1).reset_index(drop=True)
    ],axis=1).set_index(["geneID","gene_name","countbinID"])
    
    return table

In [130]:
exp_table = read_expression_table('jscs/allGenes.expression.data.txt.gz')

exp_table.index.get_level_values(1)
exp_genes = set(exp_table.index.get_level_values(1))

In [260]:
len(exp_genes)

8093

In [160]:
splice_binary_table = pd.DataFrame(
    {"spliced":np.zeros(len(exp_genes))},
    index=exp_genes, dtype=int
)

In [261]:
splice_binary_table

,spliced
TRAPPC12,0
CCDC6,0
SEL1L3,0
PDXK,0
H1FX-AS1,0
...,...
SLC39A10,0
SERP1,0
BET1L,0
RAPGEF1,0


In [163]:
splice_binary_table.loc[genewise_genes,:] = 1

In [172]:
splice_binary_table.to_csv("splice_binary_table.txt",sep='\t')

## ChIP-seq-data

In [199]:
ls ChIP-seq-data/bed_files/

GFP_1_DOX_pos_IDX1.rep1_sorted_peaks.narrowPeak.bed
GFP_2_DOX_pos_IDX2.rep1_sorted_peaks.narrowPeak.bed
s303_1_DOX_neg_IDX13.rep1_sorted_peaks.narrowPeak.bed
s303_1_DOX_pos_IDX7.rep1_sorted_peaks.narrowPeak.bed
s303_2_DOX_neg_IDX14.rep1_sorted_peaks.narrowPeak.bed
s303_2_DOX_pos_IDX8.rep1_sorted_peaks.narrowPeak.bed
sox9_1_DOX_neg_IDX11.rep1_sorted_peaks.narrowPeak.bed
SOX9_1_DOX_pos_IDX3.rep1_sorted_peaks.narrowPeak.bed
sox9_2_DOX_neg_IDX12.rep1_sorted_peaks.narrowPeak.bed
SOX9_2_DOX_pos_IDX4.rep1_sorted_peaks.narrowPeak.bed


In [234]:
%%bash 
GTF=~/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf

for f in ChIP-seq-data/bed_files/*.bed; do 
    o=${f/_IDX*/.txt}

    echo "================================================================"
    echo $o

    cat $GTF \
    | awk -v FS='\t' '$3=="gene"' \
    | bedtools intersect -wa -wb -a - -b $f \
    | cut -f 9,18 | uniq | awk -v FS='\t' '{print $1,$2}' \
    | awk -v FS=";" -v OFS="" '{gsub("\"",""); print $3}' \
    | awk -v FS=' ' -v OFS="" '{print $2}' > $o

done


ChIP-seq-data/bed_files/GFP_1_DOX_pos.txt
ChIP-seq-data/bed_files/GFP_2_DOX_pos.txt
ChIP-seq-data/bed_files/s303_1_DOX_neg.txt
ChIP-seq-data/bed_files/s303_1_DOX_pos.txt
ChIP-seq-data/bed_files/s303_2_DOX_neg.txt
ChIP-seq-data/bed_files/s303_2_DOX_pos.txt
ChIP-seq-data/bed_files/sox9_1_DOX_neg.txt
ChIP-seq-data/bed_files/SOX9_1_DOX_pos.txt
ChIP-seq-data/bed_files/sox9_2_DOX_neg.txt
ChIP-seq-data/bed_files/SOX9_2_DOX_pos.txt


In [239]:
%%bash 
cat \
ChIP-seq-data/bed_files/SOX9_1_DOX_pos.txt \
ChIP-seq-data/bed_files/SOX9_2_DOX_pos.txt \
| uniq > SOX9_DOX_pos.txt
# ChIP-seq-data/bed_files/sox9_1_DOX_neg.txt \
# ChIP-seq-data/bed_files/sox9_2_DOX_neg.txt \

In [266]:
cat ChIP-seq-data/bed_files/sox9_1_DOX_neg.txt

POMK
ZNF37BP
LINC00662


In [268]:
# cat SOX9_DOX_pos.txt

In [257]:
%%bash 
export TEISERDIR='/data_gilbert/home/aarab/Workflows/TEISERv1.1'
exp="splice_binary_table.txt"

    gene="SOX9_DOX_pos.txt"

    perl ${TEISERDIR}/run_mi_gene_list.pl \
        --expfile=$exp \
        --genefile=$gene \
        --exptype=discrete \
        --species=human \
        --doremovedups=0 \
        --doremoveextra=0 
        # --ebins=11 \

    perl /data_gilbert/home/aarab/Workflows/TEISERv1.1/Scripts/teiser_draw_matrix.pl \
        --pvmatrixfile=splice_binary_table.txt_GENESET/splice_binary_table.txt.matrix \
        --summaryfile=splice_binary_table.txt_GENESET/splice_binary_table.txt.summary \
        --expfile=splice_binary_table.txt_GENESET/splice_binary_table.txt \
        --quantized=1 \
        --colmap=/data_gilbert/home/aarab/Workflows/TEISERv1.1/Scripts/HEATMAPS/cmap_1.txt --order=0 --min=-3 --max=3 --cluster=5 

    out=${gene/.txt/}
    mv ${exp}_GENESET $out

The TEISERDIR environment variable is /data_gilbert/home/aarab/Workflows/TEISERv1.1
Thu Jan 20 20:20:24 PST 2022
Remove duplicates, create splice_binary_table.txt_GENESET/splice_binary_table.txt
step 2: seed optimization.
Expfile loaded: 8093 values...
Quantizing the input vector...Done
Number of clusters: 2
calculating the p-value matrix.
Allocating memory ... Done
doing stats: mi = 0.000616
pass = 0.037600
z = 3.717622
freq 0: 0.047744	383	8022	383	8093
freq 1: 0.000000	0	71	383	8093
step 7: drawing matrix.
Reading MI data ... Done.
Start drawing
0
Outputing EPS file splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.eps
Convert to PDF splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.eps splice_binary_table.txt_GENESET/splice_binary_table.txt.summary.pdf
Finished.
Reading MI data ... Done.
Start drawing
0
Outputing EPS file splice_binary_table.txt_G

Option suffix requires an argument


In [193]:
ls /data_gilbert/home/aarab/iPAGE/run_mi_gene_list.pl

/data_gilbert/home/aarab/iPAGE/run_mi_gene_list.pl


In [183]:
!echo ${TEISERDIR}

In [80]:
!date

Thu Jan 13 19:49:49 PST 2022
